# Budget optimizer based on cost curves

In [2]:
#@title Import Libraries
# import libraries
import numpy as np
import scipy.optimize as sco

In [3]:
#@title Setup
# setup variables
# media_budget = 10000 # total media budget
media_labels = ['Facebook',   'Google Search',  'Google Brand',   'Pinterest',      'TikTok' ] # channel names
media_coef1 = [-0.5,            - 0.2,            -0.1,             -0.1,             -0.01] 
media_coef2 = [0.1,             0.0,                0.1,             0.06,             0.01]
media_const =[0,        10,               20,               5,                10 ]

const = 0

In [1]:
#@title Enter Budget
media_budget = input("Enter budget:")
 
# type cast into integer
media_budget = int(media_budget)

Enter budget:100000


In [4]:
#@title Function
# the function for our model
def model_function(x, media_coef1, media_coef2, media_const, const):
    # transform variables and multiply them by coefficients to get contributions
    channel_1_contrib = media_coef1[0] * pow(x[0],2) + media_coef2[0] * x[0] + media_const [0]
    channel_2_contrib = media_coef1[1] * pow(x[1],2) + media_coef2[1] * x[1] + media_const [1]
    channel_3_contrib = media_coef1[2] * pow(x[2],2) + media_coef2[2] * x[2] + media_const [2]
    channel_4_contrib = media_coef1[3] * pow(x[3],2) + media_coef2[3] * x[3] + media_const [3]
    channel_5_contrib = media_coef1[4] * pow(x[4],2) + media_coef2[4] * x[4] + media_const [4]

    # sum contributions and add constant
    y = channel_1_contrib + channel_2_contrib + channel_3_contrib + channel_4_contrib + channel_5_contrib + const 
    # return negative conversions for the minimize function to work
    return -y 

In [5]:
#@title Constraints
# set up guesses, constraints and bounds
num_media_vars = len(media_labels)
guesses = num_media_vars*[media_budget/num_media_vars,] # starting guesses: divide budget evenly

args = (media_coef1, media_coef2, media_const, const) # pass non-optimized values into model_function

con_1 = {'type': 'eq', 'fun': lambda x: np.sum(x) - media_budget} # so we can't go over budget
constraints = (con_1)

bound = (0, media_budget) # spend for a channel can't be negative or higher than budget
bounds = tuple(bound for x in range(5)) # range = number of channels



In [6]:
#@title Run Optimizer
# run the SciPy Optimizer
solution = sco.minimize(model_function, x0=guesses, args=args, method='SLSQP', constraints=constraints, bounds=bounds)

In [7]:
#@title Print Solution
# print out the solution
print(f"Spend: ${round(int(media_budget),0)}\n")
# print(f"Optimized CPA: ${round(media_budget/(-1 * solution.fun),0)}")
print("Allocation:")
for i in range(len(media_labels)):
    print(f"  -{media_labels[i]}: ${round(int(solution.x[i]),0)} ({round(int(solution.x[i]/media_budget*100),0)}%)")

Spend: $100000

Allocation:
  -Facebook: $1574 (1%)
  -Google Search: $3936 (3%)
  -Google Brand: $7873 (7%)
  -Pinterest: $7874 (7%)
  -TikTok: $78740 (78%)
